In [1]:
import pandas as pd
import numpy as np
import os 
from utils.dataframe_tools import filter_out_nan 

In [2]:
df = pd.read_csv('./Data/Test/merge_tls_test_01.csv')

In [3]:
list_col = df.columns.tolist()

In [9]:
# def protocol_tree(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
#     """
#     Find the hierarchy structure of protocols by handling the csv columns. 
#     """
#     dict_protocol_tree = {item: [] for item in list_layers} 
#     dict_protocol_tree['statistics'] = [] 
#     len_prefix = 1 # length of current prefix, i.e. eth
#     for field in list_fields: # handle statistic information  
#         part_field = field.split('.') 
#         len_field = len(part_field) 
#         prefix = part_field[0] # layer: eth, ip, tcp, tls
#         if prefix not in list_layers and len_field == 1: 
#             dict_protocol_tree['statistics'].append(field) 
#         elif field.startswith(prefix) and len_field - len_prefix == 1: # find fields like xx.xx and input as values
#             dict_protocol_tree[prefix].append(field) 
#     len_prefix += 1 # xx.xx 
#     list_prefix = [] # xx.xx
#     for layer in list_layers: 
#         # list_prefix = [] # xx.xx
#         for prefix in dict_protocol_tree[layer]: # {layer: [prefix]}, find these keys already in the dict 
#             for field in list_fields: 
#                 if field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: # find fields like xx.xx.xx and input as values
#                     if prefix not in dict_protocol_tree: 
#                         dict_protocol_tree[prefix] = [] 
#                         dict_protocol_tree[prefix].append(field)
#                     else: 
#                         dict_protocol_tree[prefix].append(field) 
#             list_prefix.append(prefix) 
#     len_prefix += 1 # xx.xx
#     for new_layer in list_prefix: # [new_layer, ...] 
#         if new_layer in dict_protocol_tree:
#             for prefix in dict_protocol_tree[new_layer]: # {new_layer: [prefix]}
#                 for field in list_fields: # scan the whole column list to find the fields which statify the prefix 
#                     if field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
#                         if prefix not in dict_protocol_tree: 
#                             dict_protocol_tree[prefix] = [] 
#                             dict_protocol_tree[prefix].append(field)
#                         else: 
#                             dict_protocol_tree[prefix].append(field) 
#     return dict_protocol_tree

In [34]:
def find_fields_by_prefix(layers: list, layer_prefixes: dict, fields: list, len_prefix: int, init: bool): 
    next_layers = []
    if len(layers) > 0: 
        if init: 
            for field in fields: 
                prefix = field.split('.')[0] 
                if prefix not in layers and len(field.split('.')) - len_prefix == 0: 
                    layer_prefixes['statistics'].append(field) 
                elif field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                    layer_prefixes[prefix].append(field) 
                    next_layers.append(field) 
        else: 
            for prefix in layers: 
                # for prefix in layer_prefixes[layer]: 
                for field in fields: 
                    if field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                        if prefix not in layer_prefixes: 
                            layer_prefixes[prefix] = [] 
                            layer_prefixes[prefix].append(field) 
                        else: 
                            layer_prefixes[prefix].append(field) 
                    else: 
                        continue 
                    next_layers.append(field) 
            # len_prefix += 1 
    init = False
    return layer_prefixes, next_layers, init 

In [35]:
def protocol_tree(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    """
    Find the hierarchy structure of protocols by handling the csv columns. 
    """
    dict_protocol_tree = {item: [] for item in list_layers} 
    dict_protocol_tree['statistics'] = [] 
    lens = [len(item.split('.')) for item in list_fields]
    len_prefix = 1 # length of current prefix, i.e. eth 
    max_field_len = max(lens)
    init = True
    while len_prefix < max_field_len: 
        # len_prefix += 1 # xx.xx 
        # list_prefix = [] # xx.xx 
        dict_protocol_tree, list_layers, init = find_fields_by_prefix(list_layers, dict_protocol_tree, list_fields, len_prefix, init) 
        len_prefix += 1
    return dict_protocol_tree 

In [36]:
dict_tree = protocol_tree(list_col) 
print(dict_tree)

{'eth': ['eth.dst', 'eth.src', 'eth.type', 'eth.padding'], 'ip': ['ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.len', 'ip.id', 'ip.flags', 'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.src', 'ip.dst'], 'tcp': ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.seq_raw', 'tcp.ack', 'tcp.ack_raw', 'tcp.hdr_len', 'tcp.flags', 'tcp.window_size_value', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.checksum', 'tcp.urgent_pointer', 'tcp.options', 'tcp.option_kind', 'tcp.option_len'], 'tls': ['tls.'], 'statistics': ['frame_num', 'reassembled_segments'], 'eth.dst': ['eth.dst.lg', 'eth.dst.ig'], 'eth.src': ['eth.src.lg', 'eth.src.ig'], 'ip.dsfield': ['ip.dsfield.dscp', 'ip.dsfield.ecn'], 'ip.flags': ['ip.flags.rb', 'ip.flags.df', 'ip.flags.mf'], 'tcp.flags': ['tcp.flags.res', 'tcp.flags.ae', 'tcp.flags.cwr', 'tcp.flags.ece', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.flags.str'], 'tcp.opt

In [33]:
def find_fields_by_prefix(layers: list, layer_prefixes: dict, fields: list, len_prefix: int, init: bool):
    # Initialize next_layers
    next_layers = []
    
    # Ensure 'statistics' exists
    if 'statistics' not in layer_prefixes:
        layer_prefixes['statistics'] = []
    
    if len(layers) > 0:
        if init:
            # Initial phase: Assign fields to top-level layers or statistics
            for prefix in layers:
                if prefix not in layer_prefixes:
                    layer_prefixes[prefix] = []
                for field in fields:
                    field_parts = field.split('.')
                    if not field.startswith(prefix + '.') and len(field_parts) == len_prefix:
                        layer_prefixes['statistics'].append(field)
                    elif field.startswith(prefix + '.') and len(field_parts) == len_prefix + 1:
                        layer_prefixes[prefix].append(field)
                        # Add the next prefix to next_layers (e.g., 'dst' from 'eth.dst')
                        next_prefix = field_parts[len_prefix]
                        if next_prefix not in next_layers:
                            next_layers.append(next_prefix)
        else:
            # Non-initial phase: Process deeper layers
            for layer in layers:
                if layer not in layer_prefixes:
                    continue
                for prefix in layer_prefixes[layer]:
                    for field in fields:
                        # Check if field starts with prefix and has the correct depth
                        if field.startswith(prefix + '.') and len(field.split('.')) == len_prefix + 1:
                            field_parts = field.split('.')
                            next_prefix = field_parts[len_prefix]
                            if next_prefix not in layer_prefixes:
                                layer_prefixes[next_prefix] = []
                            layer_prefixes[next_prefix].append(field)
                            if next_prefix not in next_layers:
                                next_layers.append(next_prefix)
        len_prefix += 1
    else:
        # If layers is empty, ensure loop can exit
        len_prefix += 1
    
    init = False
    return layer_prefixes, len_prefix, next_layers, init

def protocol_tree(list_fields: list, list_layers=['eth', 'ip', 'tcp', 'tls']):
    """
    Find the hierarchy structure of protocols by handling the csv columns.
    """
    if not list_fields:
        return {'statistics': [], **{layer: [] for layer in list_layers}}

    # Initialize dictionary
    dict_protocol_tree = {item: [] for item in list_layers}
    dict_protocol_tree['statistics'] = []
    
    # Calculate max field depth
    lens = [len(item.split('.')) for item in list_fields]
    max_field_len = max(lens, default=1)
    
    len_prefix = 1  # Start with top-level prefixes (e.g., 'eth')
    init = True
    list_current_layers = list_layers
    
    while len_prefix <= max_field_len and list_current_layers:
        dict_protocol_tree, len_prefix, next_layers, init = find_fields_by_prefix(
            list_current_layers, dict_protocol_tree, list_fields, len_prefix, init
        )
        list_current_layers = next_layers  # Update layers for next iteration
    
    return dict_protocol_tree

# Test the function
dict_tree = protocol_tree(list_col)
print(dict_tree)

{'eth': ['eth.dst', 'eth.src', 'eth.type', 'eth.padding'], 'ip': ['ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.len', 'ip.id', 'ip.flags', 'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.src', 'ip.dst'], 'tcp': ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.seq_raw', 'tcp.ack', 'tcp.ack_raw', 'tcp.hdr_len', 'tcp.flags', 'tcp.window_size_value', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.checksum', 'tcp.urgent_pointer', 'tcp.options', 'tcp.option_kind', 'tcp.option_len'], 'tls': ['tls.'], 'statistics': ['frame_num', 'reassembled_segments', 'frame_num', 'reassembled_segments', 'frame_num', 'reassembled_segments', 'frame_num', 'reassembled_segments']}


In [7]:
test = 'frame_num' 
test1 = 'eth.dst.ig' 
test2 = 'eth.dst'
a = test.split('.') 
b = test1.split('.') 
c = test2.split('.')
print(a) 
print(b)
print(c)
print(c in b)

['frame_num']
['eth', 'dst', 'ig']
['eth', 'dst']
False


In [ ]:
def protocol_tree_extract(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    dict_fields = {} 
    for layer in list_layers: 
        for field in list_fields: 

    for field in list_fields: 
        s_field = field.split('.') 
        if len(s_field) == 1 and s_field[0] not in list_layers: # statistical information, like frame_num, reassembled_info 
            dict_fields['statistics'].append(s_field) 
            
        if s_field[0] in list_layers: # layer prefix 
            if s_field[0] not in dict_fields.keys(): 
                last_len = len(s_field) 
                dict_fields[s_field[0]].append(s_field) 
            

    return 

In [3]:
all_nan_cols = df.columns[df.isna().all()] 

In [6]:
df_nonan = df.drop(columns=all_nan_cols)

In [2]:
path_test_directory = './Data/Test/filter_nan' 
# path_tls_csv = os.path.join(path_test_directory, 'tls_test_01.pcapng.csv') 
# path_tls_reassemble_csv = os.path.join(path_test_directory, 'reassemble_tls_test_01.pcapng.csv') 
path_test_csv = os.path.join(path_test_directory, 'filter_nan_merge_tls_test_01.csv') 

In [3]:
df = pd.read_csv(path_test_csv, index_col='frame_num') 
df.shape 

(1959, 190)

In [4]:
mask_df = df.notnull() 

In [21]:
dict_true = {} 
for col_num in range(mask_df.shape[1]): 
    list_true_indices = list(mask_df[mask_df.iloc[:, col_num]].index) 
    dict_true[col_num] = list_true_indices  

# for col in mask_df.columns: 
#     list_true_indices = list(mask_df[mask_df.loc[:, col]].index) 
#     dict_true[col] = list_true_indices 

In [22]:
# continuous
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
last_key = 0
last_value = dict_true[last_key] 
list_col = []
for key, value in dict_true.items(): 
    if key == last_key: # init 
        dict_block['block'].append(block_flag) 
        list_col.append(key) 
        dict_block['columns'].append(list_col.copy()) 
        dict_block['rows'].append(dict_true[key]) 
    else: 
        if value == last_value: 
            dict_block['columns'][block_flag].append(key) 
        if value != last_value: 
            block_flag += 1 
            dict_block['block'].append(block_flag) 
            list_col.clear() 
            list_col.append(key) 
            dict_block['columns'].append(list_col.copy()) 
            dict_block['rows'].append(dict_true[key]) 
            last_key = key 
            last_value =value 


In [7]:
df_block = pd.DataFrame(dict_block)

In [23]:
from tqdm import tqdm 

block_values = dict_block['block'] 
columns_values = dict_block['columns'] 
rows_values = dict_block['rows'] 

list_sub_df = []

for block_name, columns, rows in tqdm(zip(block_values, columns_values, rows_values)): 
    subset_rows = df.loc[rows]
    sub_df = subset_rows.iloc[:, columns]
    list_sub_df.append(sub_df) 

41it [00:00, 826.97it/s]


In [7]:
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
list_col = [] 
list_record_col = []
for key, value in dict_true.items(): 
    if key not in list_record_col: 
        for ik, iv in dict_true.items(): 
            if iv == value: 
                if block_flag not in dict_block['block']: 
                    dict_block['block'].append(block_flag) 
                    dict_block['rows'].append(dict_true[key]) 
                list_col.append(ik) 
            if iv != value: 
                continue 
        dict_block['columns'].append(list_col.copy()) 
        list_record_col.extend(list_col.copy()) 
        list_col.clear()
        block_flag += 1 

In [ ]:
print(dict_block)

In [8]:
df_block = pd.DataFrame(dict_block)

In [9]:
true_indices = list(mask_df[mask_df['tls_handshake_server_point_len']].index) 

In [6]:
intesection_series = mask_df.all()